**Jupyter notebook**: Place your cursor in each In field and press Shift-Enter to execute it.


In [ ]:
system"cd ",getenv[`HOME],"/course-introductory-workshop"
.trn.nbdir:system"cd"
\l scripts/loaddata.q

**Learning objectives** 

To understand:
* How to construct a qSQL query
* How to assign a variable
* Useful inbuilt aggregations 
* Grouping with by
* Using `fby` to avoid nested queries
* Updating existing data 
* Temporal arithmetic 

# Data overview

The taxi database details cab fares in New York City. The data was provided from the
[NYC Taxi & Limousine Commission](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

The database we have loaded contains trip data starting at January 1st, 2009 until March 31st, 2009. The
database contains 6.9 million records and occupies 6Gb on disk with no compression. 

The taxi database is stored in a table called `trips`. To supplement the data, there is an additional New York City weather dataset called `weather`. This data was collected by the  [National Oceanic and Atmospheric Administration](https://www.noaa.gov) and distributed by the 
[National Climate and Data Center](https://www.ncdc.noaa.gov). The weather dataset contains daily
summaries for January 2009 from 187 distinct weather stations surrounding New York City.

# Data exploration

Tables form the core of Kx technology. Here we will go through how to access tables and examine their contents.

See what tables are currently in the database:

In [ ]:
tables[]

Inspect a table by typing its name.

In [ ]:
smalltrips

It is very easy to interrogate the kdb+/q process to get information from the process about the size and shape of the tables.

We can see how many records are in the table, by using [`count`](https://code.kx.com/q/ref/count).

We can also inspect the schema of `trips` with [`meta`](https://code.kx.com/q/ref/meta), obtaining a table of available columns and their types and other information. The following columns are produced:

- `c`: column name
- `t`: column [type](https://code.kx.com/q/ref/#datatypes)
- `f`: [foreign keys](https://code.kx.com/q/wp/foreign-keys/)
- `a`: [attributes](https://code.kx.com/q/basics/syntax/#attributes): modifiers applied for performance characteristics

In [ ]:
count trips
meta trips

# qSQL

If you have used SQL, you will find the syntax of qSQL queries very similar.

 <img src="images/qbies.png" width="50px" align="left"/><p style='color:#273a6e'><i> Here qSQL is like SQL but does not require a <code>*</code> to select all columns. </i></p> 

In [ ]:
select from smalltrips

Get the `vendor`, `pickup_time` and `fare` columns from the entire table.

In [ ]:
select vendor, pickup_time, fare from smalltrips

The next query extracts a few columns, but also filters on two criteria. Just as in SQL, table results can be filtered by expressions following a `where`. Multiple filter criteria, separated by `,`, are evaluated starting from the left.

Get taxi trips from the first day of data where the tip is greater than 20.

 <img src="images/qbies.png" width="50px" align="left"/><p style='color:#273a6e'><i> The first day of data is found by applying aggregation <code>min</code> to the <code>date</code> column, and selecting dates equal to it. <br><br> Q has many aggregations. We shall encounter more later. </i></p> 

In [ ]:
select date, month, vendor, passengers, fare, tip from trips 
    where date = min date, tip > 20   

**Structure of kdb+ database**

<img src="images/DbStructure.png" width="600" height="400">

Which query runs faster?

+ 1. ```select date, month, vendor, passengers, fare, tip from trips where date = min date, tip > 20```

+ 2. ```select date, month, vendor, passengers, fare, tip from trips where tip > 20, date = min date```

(1) runs faster. The `trips` table is a partitioned on the `date` column, which is the first filter in the [Where phrase](https://code.kx.com/q/basics/qsql/#where-phrase). Filtering on the partition column value first minimizes the number of directories to read.

We can test this theory with the [`\ts`](https://code.kx.com/q/basics/syscmds/#ts-time-and-space) system command.

In [ ]:
\ts select date, month, vendor, passengers, fare, tip from trips where date = min date, tip > 20
\ts select date, month, vendor, passengers, fare, tip from trips where tip > 20, date = min date

Include calculated columns in the result by assignment in the form `column_name : value`.

In [ ]:
select duration, total: fare + tip, fare, tip from trips

A virtual column `i` maps to a record index in the table. A simple aggregation can be obtained by taking the count of this virtual column.

In [ ]:
select count i from trips where date = min date, passengers = 2

Further reading:
-   [qSQL syntax](https://code.kx.com/q/basics/qsql/)
-   [*Q for Mortals* §9: Queries](https://code.kx.com/q4m3/9_Queries_q-sql/)

##### Exercise 1

- On the earliest date, how many trips had fewer than two passengers?

In [ ]:
select count i from trips where date = min date, passengers < 2
//alternative way- select count i from trips where date = first date, passengers < 2

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex1_a[] //check correct output

- What about the last date?

In [ ]:
select count i from trips where date = max date, passengers < 2
//alternative way- select count i from trips where date = last date, passengers < 2

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex1_b[] //check correct output

##### Exercise 2
- Select `payment_type` and `fare` from the trips data for the first date.


In [ ]:
select payment_type, fare from trips where date = min date

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex2[] //check correct output

## Assignment

When exploring data, storing intermediate results is useful for avoiding repeated work.

 <img src="images/qbies.png" width="50px" align="left"/><p style='color:#273a6e'><i> In q, assignment is denoted by <code>:</code>.<br><br>The equal sign <code>=</code> retains its traditional meaning of testing for equality. </i></p>

Create a new variable as follows:

```
var_name: expression
```

In order to take a closer look at only the data corresponding to January 2009, we can select the data from trips and store the resulting table in a variable called `jan09`.

In [ ]:
// Use keyword 'within' to filter the date
jan09:select from trips where date within 2009.01.10 2009.01.31

// Check how many records are in the filtered table
count jan09

 <img src="images/qbies.png" width="50px" align="left"/><p style='color:#273a6e'><i> By making this assignment, the new variable <code>jan09</code> is stored in memory. <br><br> This is true whether the source data was also in memory or stored on disk. </i></p>

##### Exercise 3

+ Assign the result of Exercise 2 to a variable called `res2`

In [ ]:
res2:select payment_type, fare from trips where date = min date;

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex3[] //check correct output

## Aggregations

With the filtered data in `jan09`, aggregations and other analytics can be run without performing the filter query again. Below is a simple aggregation returning the sum of both the `fare` and `tip` columns. 

In [ ]:
select sum fare, sum tip from jan09

[`sum`](https://code.kx.com/q/ref/sum/) is one of many built-in aggregations. Other built-in aggregations include, but are not limited to

- [`avg`](https://code.kx.com/q/ref/avg/#avg) - average (mean)
- [`med`](https://code.kx.com/q/ref/med/) - median
- [`min`](https://code.kx.com/q/ref/min/) - minimum value
- [`max`](https://code.kx.com/q/ref/max/) - maximum value
- [`count`](https://code.kx.com/q/ref/count/) - number of values

Reference: [Mathematics and statistics](https://code.kx.com/q/basics/math/)

##### Exercise 4
- Calculate the minimum and maximum tip from the `jan09` table.


In [ ]:
// maximum tip and minimum tip
select maxTip:max tip, minTip:min tip from jan09

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex4[] //check correct output

## Grouping with `by` 

Unlike SQL, qSQL lets you group and aggregate separately.
The easiest way to group similar values together is to use the `by` clause.

When used without an aggregation, `by` returns a list of values from the selected column. List types are completely valid types in q, so this fine.

In [ ]:
select fare by vendor from jan09

To extract some useful insight from the data, grouping is best used with aggregation functions as shown below:

In [ ]:
select sum fare, sum tip by vendor from jan09

In [ ]:
// Get the number of records per day in the filtered table
select count i by date from jan09

##### Exercise 5
- What is the biggest tip for each company?

In [ ]:
select max tip by vendor from jan09

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex5[] //check correct output

##### Exercise 6 
- What is the highest tip and average tip per payment_type?

In [ ]:
select maxTip:max tip, avgTip:avg tip by payment_type from jan09

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex6[] //check correct output

## Using `fby` to avoid nested queries

Nested queries are commonly required in SQL where filter criteria require aggregations in the context of some other column. For example, getting all records where the ride duration is less than the average for that taxi’s vendor.

In SQL  we would first get the average duration for each vendor; then perform a query for each duration and vendor:

In [ ]:
// Get the average duration per vendor and save resulting table in a variable
resBy: select avgDuration:avg duration by vendor from jan09

In [ ]:
// Using 'lj' to join the average duration column to our table
// Don't worry about this now, joins are discussed in a later section
select from jan09 lj resBy where duration < avgDuration

In q, this can be simply expressed using [`fby`](https://code.kx.com/q/ref/fby/).

The syntax of `fby` is `(aggregation;data) fby group`, as query below. Compare the above statement to how it was done without `fby`.

In [ ]:
select max fare from jan09 where duration < (avg;duration) fby vendor

You can see how much simpler an `fby` clause is compared to the above statements.

##### Exercise 7 

- Which payment type produces the highest average tip when only trips with a fare larger than the average for each vendor is considered?

In [ ]:
res7a:select avg tip by payment_type from jan09 where fare > (avg;fare) fby vendor;
show res7a;
select payment_type from res7a where tip = max tip 

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex7_a[] //check correct output

- Which vendor has the largest number of trips when only considering trips shorter than the average duration for each vendor?

In [ ]:
res7b:select count i by vendor from jan09 where duration < (avg;duration) fby vendor;
show res7b;
select vendor from res7b where x = max x

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex7_b[] //check correct output

## Updating existing data

When exploring, you often need to fix bad data, whether from incorrect entry, incompatible units of measure, or other reasons. 

If we inspect the data and look at the maximum passengers for each vendor, the maximum number for 2 vendors is 6.

In [ ]:
select max passengers by vendor from jan09

If the above data is wrong, it can be fixed using `update` to change the data. Below, the passengers column is set to the value `5` in any entry where the passenger count is greater than 5. This is then assigned back to `jan09`.

In [ ]:
// Find all records where the number of passengers is greater than 5
select from jan09 where passengers > 5

In [ ]:
// Update maximum number of passengers to be 5 
// Save changes by reassigning back to jan09
jan09: update passengers: 5 from jan09 where passengers > 5

In [ ]:
jan09

Observe below that the maximum passengers reported has now been updated.

In [ ]:
select max passengers by vendor from jan09

Now that we have updated the table, we can add a column with the weighted average fare per passenger:

In [ ]:
jan09:update wAvgfare:passengers wavg fare from jan09
meta jan09 //new column has been added to the end of the table

We also see a few trips that didn't take place: no recorded duration. Let’s delete these rows:

In [ ]:
count jan09  //number of records before deleting rows
jan09:delete from jan09 where duration=00:00:00.000
count jan09  //number of records after deleteing rows 

## Temporal Arithmetic

Q supports several temporal types and arithmetic between them. Writing queries involving time is simple.  

See [here](https://code.kx.com/q/ref/#datatypes) for a summary of [`datatypes`](https://code.kx.com/q/ref/#datatypes)

The `pickup_time` column in the data has a type of *timestamp*.
As an example, we could convert the `pickup_time` values to their `minute` values (including hours and minutes), and group the data based on this time frame.

In [ ]:
select pickup_time, pickup_time.second, pickup_time.minute, pickup_time.hh from jan09

 <img src="images/qbies.png" width="50px" align="left"/><p style='color:#273a6e'><i> The query below returns the total money made each minute by cabs in January. <br><br> Observe the following expression within the query: <code>sum fare + tip</code> </i></p>

This expression highlights an important feature of q syntax: **evaluation is from right-to-left**. The argument of `sum` is everything to its right, that is `fare` plus `tip. This simple rule holds everywhere; there are no priorities to remember.

In [ ]:
select total:sum fare + tip by pickup_time.minute from jan09

In [ ]:
select total:(sum fare) + tip by pickup_time.minute from jan09 

Aggregate further on time by using `xbar` to bucket the minutes into hours. Group the minutes into 60-unit buckets to produce hours:

In [ ]:
select count i by 60 xbar pickup_time.minute from jan09 where date = 2009.01.10

##### Exercise 8
- Show the largest tip for each 15-minute timespan during the month of January.

In [ ]:
select max tip by 15 xbar pickup_time.minute from jan09

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex8_a[] //check correct output

 - Break this information down by vendor.

In [ ]:
select max tip by 15 xbar pickup_time.minute,vendor from jan09

In [ ]:
// Enter your qSQL code here 

In [ ]:
ex8_b[] //check correct output